In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyreadr

!pip install pyGPGO

In [ ]:
import pandas as pd
import numpy as np
import functools
from matplotlib import pyplot as plt
import datetime
import dill
import seaborn as sns
from collections import Counter
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
import pylab
from datetime import datetime
import pyreadr
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from math import log
from math import exp

# pyGPGO

from pyGPGO.covfunc import squaredExponential
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.RandomForest import RandomForest
from pyGPGO.GPGO import GPGO
from pyGPGO.acquisition import Acquisition

# Keras
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, BatchNormalization,LSTM
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras import backend as K

## Caricamento dataset

In [ ]:
time_series = pd.read_csv('/content/drive/My Drive/Streaming Data Management and time series/time_series_dataset.csv',
                          sep=';')

In [ ]:
time_series.head(10)

In [ ]:
time_series['Year'] = time_series['Data'].apply(lambda x: int(x.split('-')[0]))

### Preprocessing 

In [ ]:
timestep=365
future=334

In [ ]:
def crea_tensore_ml(array,timestep,future):
  x=list()
  y=list()
  valori =array
  inizio=0
  fine=timestep+future
  while inizio+fine<=len(valori):
    sequenza=valori[inizio:(inizio)+fine]
    x.append(sequenza[:(len(sequenza)-future)])
    y.append(sequenza[(len(sequenza)-future):])
    inizio=inizio+1
  return np.array(x), np.array(y)

In [ ]:
train_originale = time_series[time_series['Year']<=2016]

validation_originale = time_series[time_series['Year']==2017]

test_originale = time_series[time_series['Year']==2018]

train_ultimi365 = train_originale[len(train_originale)-365:]
train_ultimi365_val = pd.concat([train_ultimi365, validation_originale], axis=0)

ultimi365 = train_ultimi365_val[365:len(train_ultimi365_val)]
test_ultimi365_val = pd.concat([ultimi365,test_originale],axis=0)

In [ ]:
train_originale.head(3)

In [ ]:
train_ultimi365_val.head(3)

In [ ]:
train_ultimi365_val.reset_index(drop=True,inplace=True)

In [ ]:
test_ultimi365_val.head(3)

In [ ]:
test_ultimi365_val.reset_index(drop=True,inplace=True)

In [ ]:
test_ultimi365_val.head(3)

In [ ]:
train_originale_dati = np.reshape(np.array(train_originale['value'].dropna()),(train_originale.shape[0],1))
train_ultimi365_val_dati = np.reshape(np.array(train_ultimi365_val['value'].dropna()),(train_ultimi365_val.shape[0],1))

test_ultimi365_val_dati = np.reshape(np.array(test_ultimi365_val['value'].dropna()),(test_ultimi365_val.shape[0],1))

#### Standard Scaler

In [ ]:
trasformer_norm = StandardScaler()
trasformer_norm.fit(train_originale_dati)

In [ ]:
train_originale_norm = trasformer_norm.transform(train_originale_dati)
train_ultimi365_val_norm = trasformer_norm.transform(train_ultimi365_val_dati)

test_ultimi365_val_norm = trasformer_norm.transform(test_ultimi365_val_dati)

In [ ]:
x_train, y_train = crea_tensore_ml(train_originale_norm,timestep,future)
x_train.shape, y_train.shape

In [ ]:
x_validation, y_validation = crea_tensore_ml(train_ultimi365_val_norm,timestep,future)
x_validation.shape, y_validation.shape

In [ ]:
x_test, y_test = crea_tensore_ml(test_ultimi365_val_norm,timestep,future)
x_test.shape, y_test.shape

In [ ]:
x_train[0][0:4]

In [ ]:
#reshape necessaria per rete neurale
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

x_validation = np.reshape(x_validation, (x_validation.shape[0], x_train.shape[1],1))

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

##  Creazione LSTM

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
x_val_performance = x_validation[0]
y_val_reale = np.ndarray.flatten(y_validation[0])
np.shape(x_val_performance)

x_val_performance = np.reshape(x_val_performance,(1,timestep,1))

y_val_reale = np.array(y_val_reale)

y_val_reale = np.reshape(y_val_reale,(y_val_reale.shape[0],1))

y_val_reale_nonorm = trasformer_norm.inverse_transform(y_val_reale)

y_val_reale_nonorm_flatten = np.ndarray.flatten(y_val_reale_nonorm)

y_val_originale = y_val_reale_nonorm_flatten
y_val_originale[0:3]

In [ ]:
model = Sequential()

model.add(LSTM(200, activation='tanh',input_shape=(x_train.shape[1],1),
               return_sequences=True,kernel_regularizer=keras.regularizers.l2(0.001),
               recurrent_regularizer=keras.regularizers.l2(0.001),
               ))

model.add(LSTM(100, activation='tanh',
               return_sequences=False,kernel_regularizer=keras.regularizers.l2(0.001),
               recurrent_regularizer=keras.regularizers.l2(0.001),
               ))
model.add(Dropout(0.2))

model.add(Dense(345,activation='relu',kernel_initializer = 'he_uniform'))
model.add(Dropout(0.2))

model.add(Dense(y_train.shape[1],activation='linear'))

model.summary()

In [ ]:
rms = keras.optimizers.RMSprop(lr = 0.005)

model.compile(optimizer= rms,
                loss='mse')

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   patience = 10,
                   restore_best_weights = False)


network_history = model.fit(x_train,y_train,validation_data=(x_validation,y_validation),shuffle=False,
                            epochs=100,batch_size=128,callbacks=[es])

In [ ]:
epoch_completed=list(range(1,len(network_history.epoch)+1))
plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(epoch_completed, network_history.history['loss'])
plt.plot(epoch_completed, network_history.history['val_loss'])
plt.legend(['Training', 'Validation'])

In [ ]:
validation_test = train_ultimi365_val.iloc[365:len(train_ultimi365_val)].reset_index(drop=True)
validation_test.iloc[0:334]

In [ ]:
x_test = x_val_performance
x_test[0][0:4]

In [ ]:
y_val_pred = model.predict(x_test)
orizzonte=range(0,np.shape(y_val_pred)[1])

y_val_pred = np.ndarray.flatten(y_val_pred)
y_val_pred = np.reshape(y_val_pred,(y_val_pred.shape[0],1))

y_val_pred_nonorm = trasformer_norm.inverse_transform(y_val_pred)
y_val_pred_nonorm_flatten = np.ndarray.flatten(y_val_pred_nonorm)

y_val_pred_originale = y_val_pred_nonorm_flatten

In [ ]:
fig= plt.figure(figsize=(35,15))

ax = fig.add_subplot()
ax.set_title('Forecast 334 giorni da 2017-01-01 a 2017-11-30', fontdict={'fontsize': 30, 'fontweight': 'medium'})

plt.plot(orizzonte, y_val_originale,c='black')
plt.plot(orizzonte,y_val_pred_originale,c='blue')

In [ ]:
np.shape(y_val_reale), np.shape(y_val_pred)

In [ ]:
mean_absolute_percentage_error(y_val_originale,y_val_pred_originale)

## AutoML rnn senza hidden layer

In [ ]:
def plot_allenamento(history):
  epoch_completed=list(range(1,len(history.epoch)+1))
  plt.figure()
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(epoch_completed, history.history['loss'])
  plt.plot(epoch_completed, history.history['val_loss'])
  plt.legend(['Training', 'Validation'])

def plot_previsione(y_val_reale,pred,mape):
  orizzonte=range(0,len(pred))
  titolo="Forecast 334 giorni | MAPE: "+str(mape)

  fig= plt.figure(figsize=(35,15))
  ax = fig.add_subplot()
  ax.set_title(titolo, fontdict={'fontsize': 30, 'fontweight': 'medium'})

  plt.plot(orizzonte, y_val_reale,c='black')
  plt.plot(orizzonte,pred,c='blue')

In [ ]:
def df_out(parametri, mape, col_names):
  df = pd.DataFrame(parametri) 
  df.columns = col_names 
  df['mape'] = mape
  return df

In [ ]:
i=0
best=0

# numero di neuroni
num_neuroni1 = [2,1024]

# RMS ottimizzatore
lr = [0.0001, 0.001]

#reg. L2
l2 = [0.000001, 0.001]

param = {
        'num_neuroni1': ('int', num_neuroni1),
        'lr': ('cont', lr),
        'reg_l2': ('cont', l2)
        }
  

iperparametri_ottimizzare = list(param.keys())

In [ ]:
def crea_compila_rnn(num_neuroni1,lr,l2):
  model = Sequential()

  print("Numero neuroni: "+str(num_neuroni1))

  '''num_neuroni1 = int(num_neuroni1)
  print("Ora: "+str(num_neuroni1))'''
  model.add(LSTM(num_neuroni1, activation='tanh',input_shape=(365,1),
                 return_sequences=False,
                 kernel_regularizer=keras.regularizers.l2(l2),
               recurrent_regularizer=keras.regularizers.l2(l2)))
  model.add(Dense(334,activation='linear'))

  # compilazione della rete
  rms = keras.optimizers.RMSprop(lr = lr)

  model.compile(optimizer= rms,
                loss='mean_squared_error')
  return model

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   patience = 10,
                   restore_best_weights = False)

def calcola_mape(num_neuroni1,lr,reg_l2):
  global i,best
  model_rnn = crea_compila_rnn(int(num_neuroni1),lr,reg_l2)

  history = model_rnn.fit(x_train,y_train,validation_data=(x_validation,y_validation),shuffle=False,
                            epochs=100,batch_size=128,callbacks=[es])
  plot_allenamento(history)

  pred = model_rnn.predict(x_test)
  pred = np.ndarray.flatten(pred)
  pred = np.reshape(pred,(pred.shape[0],1))

  pred_nonorm = trasformer_norm.inverse_transform(pred)
  pred_nonorm_flatten = np.ndarray.flatten(pred_nonorm)

  pred_originale = pred_nonorm_flatten
  
  mape= mean_absolute_percentage_error(y_val_originale,pred_originale)

  if i==0 or mape<best:
    best=mape
    print("Migliore MAPE: "+str(mape))
    model_rnn.save('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_0_hidden_layer.h5')
    i=i+1

  plot_previsione(y_val_originale,pred_originale,mape)
  return -mape

In [ ]:
np.random.seed(2020)
iniziale=5
incrementale=20

kernel_covariance = squaredExponential()
surmodel = GaussianProcess(kernel_covariance)
acq = Acquisition(mode = "ExpectedImprovement")

SMBO = GPGO(surmodel, acq, calcola_mape, param)
SMBO.run(init_evals = iniziale, max_iter = incrementale)

In [ ]:
risultati = df_out(SMBO.GP.X,SMBO.GP.y, iperparametri_ottimizzare)
risultati['iterazione'] = np.repeat(np.array(['iniziale','incrementale']),[iniziale,incrementale])

risultati['mape']=abs(risultati['mape'])
risultati

In [ ]:
risultati['num_neuroni1'] = risultati['num_neuroni1'].apply(lambda x: int(x))

In [ ]:
risultati.to_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm(365).csv')

In [ ]:
risultati.sort_values('mape',ascending=True)

## AutoML rnn 1 hidden layer (dense)

In [ ]:
i=0
best=0

# numero di neuroni
num_neuroni1 = [2,1024]
num_neuroni2 = [2,1024]

# Dropout
dropout_perc = [0.05, 0.4]

# RMS ottimizzatore
lr = [0.0001, 0.001]

#reg. L2
l2 = [0.000001, 0.001]

param = {
         'num_neuroni1': ('int', num_neuroni1),
         'num_neuroni2': ('int', num_neuroni2),
         'dropout': ('cont', dropout_perc),
         'lr': ('cont', lr),
         'reg_l2': ('cont', l2)
        }
  

iperparametri_ottimizzare = list(param.keys())

In [ ]:
def crea_compila_rnn2layer(num_neuroni1,num_neuroni2,dropout,lr,l2):
  model = Sequential()

  print("Numero neuroni1: "+str(num_neuroni1))
  print("Numero neuroni2: "+str(num_neuroni2))

  '''num_neuroni1 = int(num_neuroni1)
  print("Ora: "+str(num_neuroni1))'''
  model.add(LSTM(num_neuroni1, activation='tanh',input_shape=(365,1),
                 kernel_regularizer=keras.regularizers.l2(l2),
               recurrent_regularizer=keras.regularizers.l2(l2)))
  
  model.add(Dense(num_neuroni2,activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(334,activation='linear'))

  # compilazione della rete
  rms = keras.optimizers.RMSprop(lr = lr)

  
  model.compile(optimizer= rms,
                loss='mean_squared_error')
  return model

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   patience = 10,
                   restore_best_weights = False)

def calcola_mape_rnn2layer(num_neuroni1,num_neuroni2,dropout,lr,reg_l2):
  global i,best
  model_rnn2layer = crea_compila_rnn2layer(int(num_neuroni1),int(num_neuroni2),dropout,lr,reg_l2)

  history = model_rnn2layer.fit(x_train,y_train,validation_data=(x_validation,y_validation),shuffle=False,
                            epochs=100,batch_size=128,callbacks=[es])
  plot_allenamento(history)

  pred = model_rnn2layer.predict(x_test)
  pred = np.ndarray.flatten(pred)
  pred = np.reshape(pred,(pred.shape[0],1))

  pred_nonorm = trasformer_norm.inverse_transform(pred)
  pred_nonorm_flatten = np.ndarray.flatten(pred_nonorm)

  pred_originale = pred_nonorm_flatten

  
  mape= mean_absolute_percentage_error(y_val_originale,pred_originale)
  if i==0 or mape<best:
    best=mape
    print("Migliore MAPE: "+str(mape))
    model_rnn2layer.save('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_1_hidden_layer.h5')
    i=i+1

  plot_previsione(y_val_originale,pred_originale,mape)
  return -mape

In [ ]:
y_val_originale[0:5]

In [ ]:
validation_originale[0:5]

In [ ]:
np.random.seed(2020)
iniziale=5
incrementale=20

kernel_covariance = squaredExponential()
surmodel = GaussianProcess(kernel_covariance)
acq = Acquisition(mode = "ExpectedImprovement")

SMBO_2layer = GPGO(surmodel, acq, calcola_mape_rnn2layer, param)
SMBO_2layer.run(init_evals = iniziale, max_iter = incrementale)

In [ ]:
risultati_2layer = df_out(SMBO_2layer.GP.X,SMBO_2layer.GP.y, iperparametri_ottimizzare)
risultati_2layer['iterazione'] = np.repeat(np.array(['iniziale','incrementale']),[iniziale,incrementale])

risultati_2layer['mape']=abs(risultati_2layer['mape'])
risultati_2layer

In [ ]:
risultati_2layer['num_neuroni1'] = risultati_2layer['num_neuroni1'].apply(lambda x: int(x))
risultati_2layer['num_neuroni2'] = risultati_2layer['num_neuroni2'].apply(lambda x: int(x))

In [ ]:
risultati_2layer.to_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm_1layer_dense(365).csv')

## AutoML rnn 2 hidden layer (dense + dense)

In [ ]:
i=0
best=0

# numero di neuroni
num_neuroni1 = [2,1024]
num_neuroni2 = [2,1024]
num_neuroni3 = [2,1024]

# DropOut
dropout_perc = [0.05, 0.4]

# RMS ottimizzatore
lr = [0.0001, 0.001]

#reg. L2
l2 = [0.000001, 0.001]

param = {
        'num_neuroni1': ('int', num_neuroni1),
         'num_neuroni2': ('int', num_neuroni2),
         'num_neuroni3': ('int', num_neuroni3),
        'dropout': ('cont', dropout_perc),
        'lr': ('cont', lr),
        'reg_l2': ('cont', l2)
        }
  

iperparametri_ottimizzare = list(param.keys())

In [ ]:
def crea_compila_rnn3layer(num_neuroni1,num_neuroni2,num_neuroni3,dropout,lr,l2):
  model = Sequential()

  print("Numero neuroni1: "+str(num_neuroni1))
  print("Numero neuroni2: "+str(num_neuroni2))
  print("Numero neuroni3: "+str(num_neuroni3))

  '''num_neuroni1 = int(num_neuroni1)
  print("Ora: "+str(num_neuroni1))'''
  model.add(LSTM(num_neuroni1, activation='tanh',input_shape=(365,1),
                 kernel_regularizer=keras.regularizers.l2(l2),
               recurrent_regularizer=keras.regularizers.l2(l2)))
  model.add(Dense(num_neuroni2,activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(num_neuroni3,activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(334,activation='linear'))

  # compilazione della rete
  rms = keras.optimizers.RMSprop(lr = lr)

  
  model.compile(optimizer= rms,
                loss='mean_squared_error')
  return model

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   patience = 10,
                   restore_best_weights = True)

def calcola_mape_rnn3layer(num_neuroni1,num_neuroni2,num_neuroni3,dropout,lr,reg_l2):
  global i,best
  model_rnn3layer = crea_compila_rnn3layer(int(num_neuroni1),int(num_neuroni2),int(num_neuroni3),dropout,lr,reg_l2)

  history = model_rnn3layer.fit(x_train,y_train,validation_data=(x_validation,y_validation),shuffle=False,
                            epochs=100,batch_size=128,callbacks=[es])
  plot_allenamento(history)

  '''pred = model_rnn2layer.predict(x_val_performance)
  pred = np.ndarray.flatten(pred)
  pred = list(map(functools.partial(converti_dato_originale,min,max),pred))
  pred = np.array(pred)
'''
  pred = model_rnn3layer.predict(x_test)
  pred = np.ndarray.flatten(pred)
  pred = np.reshape(pred,(pred.shape[0],1))

  pred_nonorm = trasformer_norm.inverse_transform(pred)
  pred_nonorm_flatten = np.ndarray.flatten(pred_nonorm)

  pred_originale = pred_nonorm_flatten

  
  mape= mean_absolute_percentage_error(y_val_originale,pred_originale)

  if i==0 or mape<best:
    best=mape
    print("Migliore MAPE: "+str(mape))
    model_rnn3layer.save('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_2_hidden_layer.h5')
    i=i+1

  plot_previsione(y_val_originale,pred_originale,mape)
  return -mape

In [ ]:
np.random.seed(2020)
iniziale=5
incrementale=20

kernel_covariance = squaredExponential()
surmodel = GaussianProcess(kernel_covariance)
acq = Acquisition(mode = "ExpectedImprovement")

SMBO_3layer = GPGO(surmodel, acq, calcola_mape_rnn3layer, param)
SMBO_3layer.run(init_evals = iniziale, max_iter = incrementale)

In [ ]:
risultati_3layer = df_out(SMBO_3layer.GP.X,SMBO_3layer.GP.y, iperparametri_ottimizzare)
risultati_3layer['iterazione'] = np.repeat(np.array(['iniziale','incrementale']),[iniziale,incrementale])

risultati_3layer['mape']=abs(risultati_3layer['mape'])
risultati_3layer

In [ ]:
risultati_3layer['num_neuroni1'] = risultati_3layer['num_neuroni1'].apply(lambda x: int(x))
risultati_3layer['num_neuroni2'] = risultati_3layer['num_neuroni2'].apply(lambda x: int(x))

In [ ]:
risultati_3layer.to_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm_2layer_dense(365).csv')

## Confronto modelli

In [ ]:
x_val_performance = x_test[0]
y_val_reale = np.ndarray.flatten(y_test[0])
np.shape(x_val_performance)

x_val_performance = np.reshape(x_val_performance,(1,365,1))

y_val_reale = np.array(y_val_reale)

y_val_reale = np.reshape(y_val_reale,(y_val_reale.shape[0],1))

y_val_reale_nonorm = trasformer_norm.inverse_transform(y_val_reale)

y_val_reale_nonorm_flatten = np.ndarray.flatten(y_val_reale_nonorm)

y_val_originale = y_val_reale_nonorm_flatten
y_val_originale[0:3]

In [ ]:
test_originale.iloc[0:3]

In [ ]:
x_test = x_val_performance

### Caricamento modello migliore senza layer nascosti


In [ ]:
ris_0layer = pd.read_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm(365).csv')

In [ ]:
best_iperparametri_0layer = ris_0layer.sort_values('mape',ascending=True).iloc[0]
best_iperparametri_0layer = dict(best_iperparametri_0layer)
best_iperparametri_0layer

In [ ]:
mod_no_hidden_best = load_model('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_0_hidden_layer.h5')

In [ ]:
mod_no_hidden_best.summary()

In [ ]:
pred_1layer = mod_no_hidden_best.predict(x_test)

pred_1layer = np.ndarray.flatten(pred_1layer)
pred_1layer = np.reshape(pred_1layer,(pred_1layer.shape[0],1))

pred_1layer_nonorm = trasformer_norm.inverse_transform(pred_1layer)
pred_1layer_nonorm_flatten = np.ndarray.flatten(pred_1layer_nonorm)

pred_1layer_originale = pred_1layer_nonorm_flatten

  
mape_1= mean_absolute_percentage_error(y_val_originale,pred_1layer_originale)
mape_1

### Caricamento modello migliore 1 layer nascosto



In [ ]:
ris_1layer = pd.read_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm_1layer_dense(365).csv')

In [ ]:
best_iperparametri_1layer = ris_1layer.sort_values('mape',ascending=True).iloc[0]
best_iperparametri_1layer = dict(best_iperparametri_1layer)
best_iperparametri_1layer

In [ ]:
mod_1_hidden_best = load_model('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_1_hidden_layer.h5')

In [ ]:
mod_1_hidden_best.summary()

In [ ]:
pred_2layer = mod_1_hidden_best.predict(x_test)

pred_2layer = np.ndarray.flatten(pred_2layer)
pred_2layer = np.reshape(pred_2layer,(pred_2layer.shape[0],1))

pred_2layer_nonorm = trasformer_norm.inverse_transform(pred_2layer)
pred_2layer_nonorm_flatten = np.ndarray.flatten(pred_2layer_nonorm)

pred_2layer_originale = pred_2layer_nonorm_flatten

mape_2= mean_absolute_percentage_error(y_val_originale,pred_2layer_originale)

### Caricamento modello migliore 2 layer nascosti

In [ ]:
ris_2layer = pd.read_csv('/content/drive/My Drive/Streaming Data Management and time series/risultati_stand_1layer_lstm_2layer_dense(365).csv')

In [ ]:
best_iperparametri_2layer = ris_2layer.sort_values('mape',ascending=True).iloc[0]
best_iperparametri_2layer = dict(best_iperparametri_2layer)
best_iperparametri_2layer

In [ ]:
mod_2_hidden_best = load_model('/content/drive/My Drive/Streaming Data Management and time series/modelli_salvati/passati_365/model_best_2_hidden_layer.h5')

In [ ]:
mod_2_hidden_best.summary()

In [ ]:
pred_3layer = mod_2_hidden_best.predict(x_test)

pred_3layer = np.ndarray.flatten(pred_3layer)
pred_3layer = np.reshape(pred_3layer,(pred_3layer.shape[0],1))

pred_3layer_nonorm = trasformer_norm.inverse_transform(pred_3layer)
pred_3layer_nonorm_flatten = np.ndarray.flatten(pred_3layer_nonorm)

pred_3layer_originale = pred_3layer_nonorm_flatten

mape_3= mean_absolute_percentage_error(y_val_originale,pred_3layer_originale)

### Plot previsioni

In [ ]:
orizzonte=range(0,len(pred_1layer))
titolo="Confronto modelli migliori"

fig= plt.figure(figsize=(35,15))
ax = fig.add_subplot()
ax.set_title(titolo, fontdict={'fontsize': 30, 'fontweight': 'medium'})

plt.plot(orizzonte, y_val_originale,c='black')
plt.plot(orizzonte,pred_1layer_originale,c='blue')
plt.plot(orizzonte,pred_2layer_originale,c='red')
plt.plot(orizzonte,pred_3layer_originale,c='green')
plt.legend(['reale','predetto_rnn_1layer_best','predetto_rnn_2layer_best','predetto_rnn_3layer_best'],prop={'size': 20})

In [ ]:
print("MODELLO 1 LAYER MAPE: "+str(mape_1))
print("MODELLO 2 LAYER MAPE: "+str(mape_2))
print("MODELLO 3 LAYER MAPE: "+str(mape_3))

In [ ]:
df_ris_test_modello_migliore = pd.DataFrame(pred_3layer_originale)

df_ris_test_modello_migliore['Data'] = test_originale.iloc[:334]['Data'].values

df_ris_test_modello_migliore.columns=['value','Data']

In [ ]:
df_ris_test_modello_migliore = df_ris_test_modello_migliore[['Data','value']]

In [ ]:
df_ris_test_modello_migliore.to_csv('/content/drive/My Drive/Streaming Data Management and time series/risultato_test_rnn(365).csv')

### Confronto performance train modello migliore

In [ ]:
def crea_tensore_conv_pred(esempio):
  val_tensore= np.ndarray.flatten(esempio)
  val_tensore = np.reshape(val_tensore,(val_tensore.shape[0],1))
  val_tensore_nonorm = trasformer_norm.inverse_transform(val_tensore)
  val_tensore_nonorm_flatten = np.ndarray.flatten(val_tensore_nonorm)

  return val_tensore_nonorm_flatten

In [ ]:
def calcola_MAPE_train(pred, reale,num_esempi):
  mape_esempi=[]
  for i in range(0,num_esempi):
    pred_model= crea_tensore_conv_pred(pred[i])
    reale_model = crea_tensore_conv_pred(reale[i])
    mape = mean_absolute_percentage_error(reale_model,pred_model)
    mape_esempi.append(mape)
  return mape_esempi

In [ ]:
pred_train = mod_2_hidden_best.predict(x_train)

In [ ]:
pred_train[0].shape, y_train[0].shape

In [ ]:
ris = calcola_MAPE_train(pred_train,y_train,pred_train.shape[0])

np.mean(ris)